In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.optimize import minimize
from matplotlib.colors import TwoSlopeNorm

import itertools 

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, brier_score_loss


In [2]:
# df_results = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\ufc_da2_model_results_test.csv')
df_history = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\model_results_train_v2.csv')
df_results = pd.read_csv(r'C:\Users\jcmar\my_files\SportsBetting\data\model_results_v2.csv')

df_results['pred_winner'] = np.where(df_results['correct_pred']==1, df_results['winner'], np.abs(1-df_results['winner']))
df_results['Date'] = df_results["event_dates"].copy()
df_results.sort_values(by='Date', ascending=True).reset_index(drop=True)

df_results['prob_winner'] = df_results[['red_probs','blue_probs']].max(axis=1)
df_results['winner_odds'] = np.where(df_results['winner'] == 1, df_results['open_red'], df_results['open_blue'])
df_results['choice_fighter_bet'] = np.where(df_results['pred_winner'] == 1, df_results['open_red'], df_results['open_blue'])
df_results['Date'] = df_results["Date"].astype(str)

df_correct_pred = df_results[df_results["correct_pred"] == 1].copy() 
df_incorrect_pred = df_results[df_results["correct_pred"] == 0].copy()

pred_correct_odds = df_results[df_results["correct_pred"] == 1]['choice_fighter_bet'].value_counts().sort_index()
pred_incorrect_odds = df_results[df_results["correct_pred"] == 0]['choice_fighter_bet'].value_counts().sort_index()

df_results.head()

,Unnamed: 0,blue_probs,red_probs,pred_winner,winner,correct_pred,event_dates,open_red,open_blue,close1_red,close1_blue,close2_red,close2_blue,red_fighter,blue_fighter,Date,prob_winner,winner_odds,choice_fighter_bet
0,0,0.543981,0.456019,0,0,1,2023-08-12,163.0,-200.0,164.0,-225.0,180.0,-198.0,josh parisian,martin buday,2023-08-12,0.543981,-200.0,-200.0
1,1,0.570416,0.429584,0,0,1,2023-08-12,300.0,-400.0,260.0,-450.0,350.0,-335.0,jp buys,marcus mcghee,2023-08-12,0.570416,-400.0,-400.0
2,2,0.788572,0.211428,0,0,1,2023-08-12,163.0,-200.0,158.0,-200.0,170.0,-188.0,polyana viana,iasmin lucindo,2023-08-12,0.788572,-200.0,-200.0
3,3,0.443205,0.556795,1,1,1,2023-08-12,-225.0,175.0,-200.0,140.0,-170.0,170.0,khalil rountree jr,chris daukaus,2023-08-12,0.556795,-225.0,-225.0
4,4,0.727696,0.272304,0,1,0,2023-08-12,200.0,-250.0,175.0,-320.0,250.0,-225.0,cub swanson,hakeem dawodu,2023-08-12,0.727696,200.0,-250.0


In [3]:
def american_to_decimal(odds):
    return np.where(odds > 0, odds / 100 + 1, 100 / np.abs(odds) + 1)

df_history['dec_open_red'] = american_to_decimal(df_history['open_red'])
df_history['dec_close1_red'] = american_to_decimal(df_history['close1_red'])
df_history['dec_close2_red'] = american_to_decimal(df_history['close2_red'])

df_history['dec_open_blue'] = american_to_decimal(df_history['open_blue'])
df_history['dec_close1_blue'] = american_to_decimal(df_history['close1_blue'])
df_history['dec_close2_blue'] = american_to_decimal(df_history['close2_blue'])

df_results['dec_open_red'] = american_to_decimal(df_results['open_red'])
df_results['dec_close1_red'] = american_to_decimal(df_results['close1_red'])
df_results['dec_close2_red'] = american_to_decimal(df_results['close2_red'])

df_results['dec_open_blue'] = american_to_decimal(df_results['open_blue'])
df_results['dec_close1_blue'] = american_to_decimal(df_results['close1_blue'])
df_results['dec_close2_blue'] = american_to_decimal(df_results['close2_blue'])

In [4]:
def implied_probs_from_decimal(odds):
    """Vectorized implied probabilities from decimal odds."""
    odds = np.asarray(odds, dtype=float)
    return 1.0 / odds
from math import isfinite

def devig_normalize(probs):
    """
    Simple normalization (a.k.a. proportional scaling).
    Works for 2+ outcomes. Sum of probs becomes 1.
    """
    probs = np.asarray(probs, dtype=float)
    s = probs.sum()
    if s <= 0 or not isfinite(s):
        raise ValueError("Invalid probabilities for normalization.")
    return probs / s

def devig_power(p_imp, tol=1e-12, max_iter=200):
    """
    'Power' or 'Harville' style reweighting: find exponent alpha so that
    sum(p_i^alpha) = 1. Returns p_fair_i ∝ p_i^alpha.
    Works for 2+ outcomes. More flexible than simple normalization
    when bookmakers’ overround is not proportional.
    """
    probs = np.asarray(p_imp, dtype=float)
    if np.any(probs <= 0):
        raise ValueError("All probs must be > 0 for power devig.")
    # Binary search on alpha
    lo, hi = 0.0, 5.0  # broad range; increase hi if needed
    for _ in range(max_iter):
        mid = 0.5 * (lo + hi)
        s = np.sum(probs ** mid)
        if abs(s - 1.0) < tol:
            alpha = mid
            break
        if s > 1.0:
            # Need larger alpha to shrink sum
            alpha = None
            lo = mid
        else:
            hi = mid
    alpha = mid if alpha is None else alpha
    fair = probs ** alpha
    return fair / fair.sum()
def devig_two_way(dec_a, dec_b, method="normalize"):
    """
    Devig a two-outcome market.
    Returns fair probabilities (pA, pB) and fair decimal odds (oddsA, oddsB).
    """
    imp = implied_probs_from_decimal([dec_a, dec_b])
    if method == "normalize":
        pf = devig_normalize(imp)
    elif method == "power":
        pf = devig_power(imp)
    # elif method == "shin":
    #     pf = devig_shin(imp)
    else:
        raise ValueError("method must be one of: normalize, power, shin")
    fair_odds = 1.0 / pf
    return pf[0], pf[1], fair_odds[0], fair_odds[1], imp

def american_to_decimal(odds):
    return np.where(odds > 0, odds / 100 + 1, 100 / np.abs(odds) + 1)

def run_devig(df, red_odds_c,blue_odds_c,dec_new_red,dec_new_blue,p_new_red, p_new_blue, p_imp_red, p_imp_blue, method='power'):
    pfair_red = []
    pfair_blue = []
    dfair_red = []
    dfair_blue = []
    pimp_red = []
    pimp_blue = []

    for i, row in df.iterrows(): 
        red_odds = row[red_odds_c]
        blue_odds = row[blue_odds_c]

        pf_red, pf_blue, df_red, df_blue, imp_red_blue = devig_two_way(red_odds, blue_odds)
        pfair_red.append(pf_red)
        pfair_blue.append(pf_blue)
        dfair_red.append(df_red)
        dfair_blue.append(df_blue)
        pimp_red.append(imp_red_blue[0])
        pimp_blue.append(imp_red_blue[1])

    df[dec_new_red] = dfair_red
    df[dec_new_blue] = dfair_blue
    df[p_new_red] = pfair_red
    df[p_new_blue] = pfair_blue
    df[p_imp_red] = pimp_red
    df[p_imp_blue] = pimp_blue
    
    return df

df_results = run_devig(df_results, 'dec_open_red', 'dec_open_blue', 'dec_fair_open_red', 'dec_fair_open_blue', 'probs_fair_open_red', 'probs_fair_open_blue', 'pimp_open_red', 'pimp_open_blue')
df_results = run_devig(df_results, 'dec_close1_red', 'dec_close1_blue', 'dec_fair_close1_red', 'dec_fair_close1_blue', 'probs_fair_close1_red', 'probs_fair_close1_blue','pimp_close1_red','pimp_close1_blue')
df_results = run_devig(df_results, 'dec_close2_red', 'dec_close2_blue', 'dec_fair_close2_red', 'dec_fair_close2_blue', 'probs_fair_close2_red', 'probs_fair_close2_blue', 'pimp_close2_red','pimp_close2_blue')

df_history = run_devig(df_history, 'dec_open_red', 'dec_open_blue', 'dec_fair_open_red', 'dec_fair_open_blue', 'probs_fair_open_red', 'probs_fair_open_blue','pimp_open_red', 'pimp_open_blue')
df_history = run_devig(df_history, 'dec_close1_red', 'dec_close1_blue', 'dec_fair_close1_red', 'dec_fair_close1_blue', 'probs_fair_close1_red', 'probs_fair_close1_blue','pimp_close1_red','pimp_close1_blue')
df_history = run_devig(df_history, 'dec_close2_red', 'dec_close2_blue', 'dec_fair_close2_red', 'dec_fair_close2_blue', 'probs_fair_close2_red', 'probs_fair_close2_blue','pimp_close2_red','pimp_close2_blue')

In [5]:
from scipy.stats import beta 

def bayesian_kelly_per_fight(alpha, beta_param, odds, n_samples=20000, clip_min=0.0):
    """
    Monte Carlo evaluate the Bayesian Kelly expectation for a single fight.
    Returns expected Kelly fraction (E[f*]) and also an estimate of the variance of f*.
    """
    samples_p = beta.rvs(alpha, beta_param, size=n_samples)
    b = odds - 1.0
    f_samples = (samples_p * odds - 1.0) / b
    f_samples = np.clip(f_samples, clip_min, None)
    return f_samples.mean(), f_samples.std(ddof=1), f_samples  # mean, sd, samples

def per_fight_sigma_from_f(f, p_hat, odds):
    """
    Quick deterministic sigma using the (Bayes) fraction f and a point p_hat.
    This computes the standard deviation of log-returns assuming probability p_hat.
    (Use Monte Carlo variant below for fully accounting uncertainty.)
    """
    b = odds - 1.0
    r_win = np.log(1 + f * b)
    r_loss = np.log(1 - f)
    mu = p_hat * r_win + (1 - p_hat) * r_loss
    var = p_hat * (r_win - mu)**2 + (1 - p_hat) * (r_loss - mu)**2
    return np.sqrt(var), mu

def portfolio_bayesian_kelly(bets, N_rounds=1000, target_mdd=None, n_mc_per_fight=20000, mc_portfolio_check=False, mc_trials=2000):
    """
    bets: list of dicts, each dict has keys:
        'alpha', 'beta'  -> beta posterior params for p
        'odds'           -> decimal odds (e.g., 2.0)
        optionally 'name'
    N_rounds : horizon used for expected MDD heuristic
    target_mdd: tolerable drawdown (fraction, e.g., 0.30). If None, no scaling applied.
    n_mc_per_fight: samples per fight to compute E[f*]
    mc_portfolio_check: if True, run Monte Carlo portfolio draws to empirically check drawdowns
    mc_trials: number of portfolio Monte Carlo trials (if checking)
    """
    m = len(bets)
    f_bayes = np.zeros(m)
    f_bayes_sd = np.zeros(m)
    sigma_i = np.zeros(m)
    names = []

    # Step 1: compute Bayesian Kelly per fight (via sampling)
    for i, bet in enumerate(bets):
        a = bet['alpha']
        bparam = bet['beta']
        odds = bet['odds']
        mean_f, sd_f, f_samples = bayesian_kelly_per_fight(a, bparam, odds, n_samples=n_mc_per_fight)
        f_bayes[i] = mean_f
        f_bayes_sd[i] = sd_f
        names.append(bet.get('name', f'bet_{i}'))

        # Quick sigma estimate using posterior mean p
        p_hat = a / (a + bparam)
        sigma, _ = per_fight_sigma_from_f(mean_f, p_hat, odds)
        sigma_i[i] = sigma

    # Step 2: portfolio volatility under independent assumption
    port_vol_full = np.sqrt(np.sum((f_bayes * sigma_i)**2))

    # Step 3: compute scaling factor k to meet target MDD (if provided)
    if target_mdd is None or port_vol_full == 0:
        k = 1.0
    else:
        k = min(1.0, target_mdd / (port_vol_full * np.sqrt(2 * np.log(N_rounds))))
    f_scaled = k * f_bayes
    port_vol_scaled = k * port_vol_full
    exp_mdd_full = port_vol_full * np.sqrt(2 * np.log(N_rounds))
    exp_mdd_scaled = port_vol_scaled * np.sqrt(2 * np.log(N_rounds))

    results = {
        'names': names,
        'f_bayes': f_bayes,
        'f_bayes_sd': f_bayes_sd,
        'sigma_i': sigma_i,
        'port_vol_full': port_vol_full,
        'port_vol_scaled': port_vol_scaled,
        'k': k,
        'f_scaled': f_scaled,
        'exp_mdd_full': exp_mdd_full,
        'exp_mdd_scaled': exp_mdd_scaled
    }

    # Optional: Monte Carlo portfolio check (simulate outcomes and compute empirical MDD distribution)
    if mc_portfolio_check:
        rng = np.random.default_rng()
        draws_mdd = np.zeros(mc_trials)
        for t in range(mc_trials):
            # For a single trial, sample true p_i from posteriors once (reflecting parameter uncertainty)
            true_ps = np.array([beta.rvs(bet['alpha'], bet['beta']) for bet in bets])
            # simulate one round of simultaneous bets repeatedly for N_rounds to get trajectory
            B = 1.0  # normalized bankroll
            peak = B
            mdd = 0.0
            for _ in range(N_rounds):
                # For each fight, sample outcome given true p
                fractional_changes = 0.0
                for i, bet in enumerate(bets):
                    odds = bet['odds']
                    f_i = f_scaled[i]
                    win = rng.random() < true_ps[i]
                    if win:
                        fractional_changes += f_i * (odds - 1.0)
                    else:
                        fractional_changes -= f_i
                B = B * (1.0 + fractional_changes)
                if B <= 0:
                    B = 1e-12  # floor
                peak = max(peak, B)
                mdd = max(mdd, (peak - B) / peak)
            draws_mdd[t] = mdd
        results['mc_mdd_dist'] = draws_mdd
        results['mc_mdd_percentile'] = {
            'median': np.median(draws_mdd),
            'p90': np.percentile(draws_mdd, 90),
            'p95': np.percentile(draws_mdd, 95),
            'p99': np.percentile(draws_mdd, 99)
        }

    return results